# Efficiency - Add multiple points

| comparable algo list |
| - |
| pivot |
| delta |
| Monte Carlo |

In [ ]:
import sys
sys.path.append("..")

from sklearn import svm
import numpy as np

import dynashap

from examples.data_utils import (
    load_npy, save_npy, preprocess_data, variance, normalize
)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')

# size_choices = [10, 100, 1000, 10000]
size_choices = [10, 100, 200]
m_choices = [10, 15, 18, 20, 22, 25, 28, 30]
stone_m = 100

for size in size_choices:
    x_train, y_train, x_test, y_test, \
    columns_name = preprocess_data(
        't_train_' + str(size) + 'p.csv',
        't_test_' + str(size) + 'p.csv')

    # process data
    x_train_ = x_train[:-2]
    y_train_ = y_train[:-2]

    # for add
    add_points_x = x_train[-2:, :]
    add_points_y = y_train[-2:]

    x_test_ = x_test
    y_test_ = y_test

    # get a init sv
    init_sv = dynashap.mc_shap(x_train_, y_train_, x_test_, y_test_, model,
                               stone_m * len(y_train_), proc_num=2)
    save_npy('am_init_mc_plus_size' + str(size) + '.npy', init_sv)

    # set a stone
    mc_plus = dynashap.mc_shap(x_train, y_train, x_test_, y_test_, model,
                               stone_m * len(y_train), proc_num=2)
    save_npy('am_mc_plus_size' + str(size) + '.npy', mc_plus)

    # mc -> 比较标准
    mc = dynashap.mc_shap(x_train, y_train, x_test_, y_test_, model, mc_stone * len(y_train))
    save_npy('am_mc'+ str(size) + 'm' + str(mc_stone) + '.npy', mc)

    # half 需要提前算好的lsv
    half_shap = dynashap.PivotShap(x_train_, y_train_, x_test_, y_test_, model,
                                  None)
    lsv = half_shap.prepare(m=len(y_train_)*stone_m)
    save_npy('am_half_lsv_size' + str(size) + '.npy', lsv)

    # # walk the m choices
    for m in m_choices:
        # delta
        delta_shap = dynashap.DeltaShap(x_train_, y_train_, x_test_, y_test_, model,
                                        init_sv)

        m_delta_sv = delta_shap.add_single_point(add_points_x[0],
                                                 add_points_y[0],
                                                 m=(len(y_train_)+1)*m)

        m_delta_sv = dynashap.utils.eval_utility(x_train[:-1], y_train[-1], x_test_, y_test_, model)/ \
                     (np.sum(m_delta_sv)* m_delta_sv)

        n_delta_shap = dynashap.DeltaShap(x_train[:-1], y_train[-1], x_test_, y_test_,
                                          model, m_delta_sv)

        delta_sv = n_delta_shap.add_single_point(add_points_x[1],
                                                 add_points_y[1],
                                                 m=(len(y_train_)+2)*m)
        save_npy('am_delta_size' + str(size) +'m' + str(m) + '.npy', delta_sv)

        # half
        half_shap = dynashap.PivotShap(x_train_, y_train_, x_test_, y_test_, model,
                                      None)
        half_shap.left_sv = lsv
        _ = half_shap.add_single_point(add_points_x[0], add_points_y[0],
                                       (len(y_train_)+1)*m,
                                       {'flag_lsv': True})
        half_shap.x_train = x_train[:-1,:]
        half_shap.y_train = y_train[:-1]
        half_sv = half_shap.add_single_point(add_points_x[1], add_points_y[1],
                                       (len(y_train_)+2)*m,
                                       {'flag_lsv': True})
        save_npy('am_half_size' + str(size) +'m' + str(m) + '.npy', half_sv)